In [15]:
# import everything and define a test runner function
from importlib import reload
from helper import run_test

import ecc
import helper
import script
import tx

### Exercise 1

#### 1.1. Make [this test](/edit/session5/tx.py) pass
```
tx.py:TxTest::test_verify_input
```

In [16]:
# Exercise 1.1

reload(tx)
run_test(tx.TxTest('test_verify_input'))

.
----------------------------------------------------------------------
Ran 1 test in 0.706s

OK


In [17]:
# Transaction Construction Example

from ecc import PrivateKey
from helper import decode_base58, p2pkh_script, SIGHASH_ALL
from script import Script
from tx import TxIn, TxOut, Tx

# Step 1
tx_ins = []
prev_tx = bytes.fromhex('0025bc3c0fa8b7eb55b9437fdbd016870d18e0df0ace7bc9864efc38414147c8')
tx_ins.append(TxIn(
            prev_tx=prev_tx,
            prev_index=0,
            script_sig=b'',
            sequence=0xffffffff,
        ))

# Step 2
tx_outs = []
h160 = decode_base58('mzx5YhAH9kNHtcN481u6WkjeHjYtVeKVh2')
tx_outs.append(TxOut(
    amount=int(0.99*100000000),
    script_pubkey=p2pkh_script(h160),
))
h160 = decode_base58('mnrVtF8DWjMu839VW3rBfgYaAfKk8983Xf')
tx_outs.append(TxOut(
    amount=int(0.1*100000000),
    script_pubkey=p2pkh_script(h160),
))
tx_obj = Tx(version=1, tx_ins=tx_ins, tx_outs=tx_outs, locktime=0, testnet=True)

# Step 3
hash_type = SIGHASH_ALL
z = tx_obj.sig_hash(0, hash_type)
pk = PrivateKey(secret=8675309)
der = pk.sign(z).der()
sig = der + bytes([hash_type])
sec = pk.point.sec()
script_sig = bytes([len(sig)]) + sig + bytes([len(sec)]) + sec
script_sig = bytes([len(script_sig)]) + script_sig
tx_obj.tx_ins[0].script_sig = Script.parse(script_sig)
print(tx_obj.serialize().hex())

0100000001c847414138fc4e86c97bce0adfe0180d8716d0db7f43b955ebb7a80f3cbc2500000000006c6b483045022100eaddb8e76617ac808d68f14370f29f4da1f33d7008519be1887992d4d6223d7f022047f35cf581579383176371ac765352347bdae791524f00142f622861f73e7161012103935581e52c354cd2f484fe8ed83af7a3097005b2f9c60bff71d35bd795f54b67ffffffff02c09ee605000000001976a914d52ad7ca9b3d096a38e752c2018e6fbc40cdf26f88ac80969800000000001976a914507b27411ccf7f16f10297de6cef3f291623eddf88ac00000000


### Exercise 2

#### 2.1. Make [this test](/edit/session5/tx.py) pass
```
tx.py:TxTest:test_sign_input
```

In [19]:
# Exercise 2.1

reload(tx)
run_test(tx.TxTest('test_sign_input'))

.
----------------------------------------------------------------------
Ran 1 test in 0.838s

OK


### Exercise 3

#### 3.1. Send 0.02 TBTC to this address `mnrVtF8DWjMu839VW3rBfgYaAfKk8983Xf`

#### Go here to send your transaction: https://testnet.blockexplorer.com/tx/send

In [34]:
# Exercise 3.1

from ecc import PrivateKey
from helper import decode_base58, p2pkh_script, SIGHASH_ALL
from script import Script
from tx import TxIn, TxOut, Tx

prev_tx = bytes.fromhex('dd672865e897902f4e39dc338e8d1c2d04470d7d3d936b1d500630c9ee322e98')
prev_index = 20
target_address = 'mnrVtF8DWjMu839VW3rBfgYaAfKk8983Xf'
target_amount = 0.02
# change_address = 'moodNF6kJjoAJhWB2HHKsFfS5726Qn3shz'
# change_amount = 0.01
# secret = 51895635635189563563
change_address = 'muwEbVxBqkHBDPwZUDe8UkWdzBhMBD21dy'
change_amount = 0.01
secret = 184582578101252000
priv = PrivateKey(secret=secret)

# initialize inputs
tx_ins = []
# create a new tx input
tx_ins.append(TxIn(prev_tx,
                   prev_index,
                   b'',
                   0xffffffff))
# initialize outputs
tx_outs = []
# decode the hash160 from the target address
d_h160 = decode_base58(target_address)
# convert hash160 to p2pkh script
script_pubkey = p2pkh_script(d_h160)
# convert target amount to satoshis (multiply by 100 million)
target_satoshis = int(target_amount*100000000)

# create a new tx output for target
tx_outs.append(TxOut(
        target_satoshis,
        script_pubkey))
# decode the hash160 from the change address
d_h160 = decode_base58(change_address)
# convert hash160 to p2pkh script
script_pubkey = p2pkh_script(d_h160)
# convert target amount to satoshis (multiply by 100 million)
change_satoshis = int(change_amount*100000000)
# create a new tx output for target
tx_outs.append(TxOut(
        change_satoshis,
        script_pubkey))
# create a new tx
tx_obj = Tx(1, tx_ins, tx_outs, 0, True)

# now sign the 0th input with the private key using SIGHASH_ALL using sign_input
tx_obj.sign_input(0, priv, SIGHASH_ALL)

# serialize and hex()
print(tx_obj.serialize().hex())

0100000001982e32eec93006501d6b933d7d0d47042d1c8d8e33dc394e2f9097e8652867dd100000006a4730440220208e17af89ac3bb9ab68b4015eb34b14c69d04d666cb2afea3c4648c8e2046f202200e8099c022fce3d99e87a216af7a834a626f31ef8eebd0ce9b8531f4d79a86000121021f215eeea28a2ef57acf880a7f414a81c36b8437a9f85782110959291e115ccbffffffff0280841e00000000001976a914507b27411ccf7f16f10297de6cef3f291623eddf88ac40420f00000000001976a9149e2959ecf770f64c5d0e4c5cef4b3b208ad350e588ac00000000


### Exercise 4

#### 4.1. Find the hash160 of the RedeemScript
```
5221022626e955ea6ea6d98850c994f9107b036b1334f18ca8830bfff1295d21cfdb702103b287eaf122eea69030a0e9feed096bed8045c8b98bec453e1ffac7fbdbd4bb7152ae
```

In [35]:
# Exercise 4.1

from helper import hash160

hex_redeem_script = '5221022626e955ea6ea6d98850c994f9107b036b1334f18ca8830bfff1295d21cfdb702103b287eaf122eea69030a0e9feed096bed8045c8b98bec453e1ffac7fbdbd4bb7152ae'

# bytes.fromhex script
b = bytes.fromhex(hex_redeem_script)
# hash160 result
print(hash160(b).hex())
# hex() to display

74d691da1574e6b3c192ecfb52cc8984ee7b6c56


In [36]:
# P2SH address construction example

from helper import encode_base58_checksum

print(encode_base58_checksum(b'\x05'+bytes.fromhex('74d691da1574e6b3c192ecfb52cc8984ee7b6c56')))

3CLoMMyuoDQTPRD3XYZtCvgvkadrAdvdXh


### Exercise 5

#### 5.1. Make [these tests](/edit/session5/helper.py) pass
```
helper.py:HelperTest:test_p2pkh_address
helper.py:HelperTest:test_p2sh_address
```

In [41]:
# Exercise 5.1

reload(helper)
run_test(helper.HelperTest('test_p2pkh_address'))
run_test(helper.HelperTest('test_p2sh_address'))

.
----------------------------------------------------------------------
Ran 1 test in 0.003s

OK
.
----------------------------------------------------------------------
Ran 1 test in 0.003s

OK


In [42]:
# z for p2sh example

from helper import double_sha256

sha = double_sha256(bytes.fromhex('0100000001868278ed6ddfb6c1ed3ad5f8181eb0c7a385aa0836f01d5e4789e6bd304d87221a000000475221022626e955ea6ea6d98850c994f9107b036b1334f18ca8830bfff1295d21cfdb702103b287eaf122eea69030a0e9feed096bed8045c8b98bec453e1ffac7fbdbd4bb7152aeffffffff04d3b11400000000001976a914904a49878c0adfc3aa05de7afad2cc15f483a56a88ac7f400900000000001976a914418327e3f3dda4cf5b9089325a4b95abdfa0334088ac722c0c00000000001976a914ba35042cfe9fc66fd35ac2224eebdafd1028ad2788acdc4ace020000000017a91474d691da1574e6b3c192ecfb52cc8984ee7b6c56870000000001000000'))
z = int.from_bytes(sha, 'big')
                    
print(sha.hex())

e71bfa115715d6fd33796948126f40a8cdd39f187e4afb03896795189fe1423c


In [43]:
# p2sh verification example

from ecc import S256Point, Signature
from helper import double_sha256

sha = double_sha256(bytes.fromhex('0100000001868278ed6ddfb6c1ed3ad5f8181eb0c7a385aa0836f01d5e4789e6bd304d87221a000000475221022626e955ea6ea6d98850c994f9107b036b1334f18ca8830bfff1295d21cfdb702103b287eaf122eea69030a0e9feed096bed8045c8b98bec453e1ffac7fbdbd4bb7152aeffffffff04d3b11400000000001976a914904a49878c0adfc3aa05de7afad2cc15f483a56a88ac7f400900000000001976a914418327e3f3dda4cf5b9089325a4b95abdfa0334088ac722c0c00000000001976a914ba35042cfe9fc66fd35ac2224eebdafd1028ad2788acdc4ace020000000017a91474d691da1574e6b3c192ecfb52cc8984ee7b6c56870000000001000000'))
z = int.from_bytes(sha, 'big')
point = S256Point.parse(bytes.fromhex('022626e955ea6ea6d98850c994f9107b036b1334f18ca8830bfff1295d21cfdb70'))
sig = Signature.parse(bytes.fromhex('3045022100dc92655fe37036f47756db8102e0d7d5e28b3beb83a8fef4f5dc0559bddfb94e02205a36d4e4e6c7fcd16658c50783e00c341609977aed3ad00937bf4ee942a89937'))
print(point.verify(z, sig))

True


### Exercise 6

#### 6.1. Validate the second signature of the first input

```
0100000001868278ed6ddfb6c1ed3ad5f8181eb0c7a385aa0836f01d5e4789e6bd304d87221a000000db00483045022100dc92655fe37036f47756db8102e0d7d5e28b3beb83a8fef4f5dc0559bddfb94e02205a36d4e4e6c7fcd16658c50783e00c341609977aed3ad00937bf4ee942a8993701483045022100da6bee3c93766232079a01639d07fa869598749729ae323eab8eef53577d611b02207bef15429dcadce2121ea07f233115c6f09034c0be68db99980b9a6c5e75402201475221022626e955ea6ea6d98850c994f9107b036b1334f18ca8830bfff1295d21cfdb702103b287eaf122eea69030a0e9feed096bed8045c8b98bec453e1ffac7fbdbd4bb7152aeffffffff04d3b11400000000001976a914904a49878c0adfc3aa05de7afad2cc15f483a56a88ac7f400900000000001976a914418327e3f3dda4cf5b9089325a4b95abdfa0334088ac722c0c00000000001976a914ba35042cfe9fc66fd35ac2224eebdafd1028ad2788acdc4ace020000000017a91474d691da1574e6b3c192ecfb52cc8984ee7b6c568700000000
```

The sec pubkey of the second signature is:
```
03b287eaf122eea69030a0e9feed096bed8045c8b98bec453e1ffac7fbdbd4bb71
```

The der signature of the second signature is:
```
3045022100da6bee3c93766232079a01639d07fa869598749729ae323eab8eef53577d611b02207bef15429dcadce2121ea07f233115c6f09034c0be68db99980b9a6c5e75402201475221022
```

The redeemScript is:
```
475221022626e955ea6ea6d98850c994f9107b036b1334f18ca8830bfff1295d21cfdb702103b287eaf122eea69030a0e9feed096bed8045c8b98bec453e1ffac7fbdbd4bb7152ae
```

In [50]:
# Exercise 6.1

from io import BytesIO
from ecc import S256Point, Signature
from helper import double_sha256, int_to_little_endian
from script import Script
from tx import Tx, SIGHASH_ALL

hex_sec = '03b287eaf122eea69030a0e9feed096bed8045c8b98bec453e1ffac7fbdbd4bb71'
hex_der = '3045022100da6bee3c93766232079a01639d07fa869598749729ae323eab8eef53577d611b02207bef15429dcadce2121ea07f233115c6f09034c0be68db99980b9a6c5e754022'
hex_redeem_script = '5221022626e955ea6ea6d98850c994f9107b036b1334f18ca8830bfff1295d21cfdb702103b287eaf122eea69030a0e9feed096bed8045c8b98bec453e1ffac7fbdbd4bb7152ae'
sec = bytes.fromhex(hex_sec)
der = bytes.fromhex(hex_der)
redeem_script = bytes.fromhex(hex_redeem_script)

hex_tx = '0100000001868278ed6ddfb6c1ed3ad5f8181eb0c7a385aa0836f01d5e4789e6bd304d87221a000000db00483045022100dc92655fe37036f47756db8102e0d7d5e28b3beb83a8fef4f5dc0559bddfb94e02205a36d4e4e6c7fcd16658c50783e00c341609977aed3ad00937bf4ee942a8993701483045022100da6bee3c93766232079a01639d07fa869598749729ae323eab8eef53577d611b02207bef15429dcadce2121ea07f233115c6f09034c0be68db99980b9a6c5e75402201475221022626e955ea6ea6d98850c994f9107b036b1334f18ca8830bfff1295d21cfdb702103b287eaf122eea69030a0e9feed096bed8045c8b98bec453e1ffac7fbdbd4bb7152aeffffffff04d3b11400000000001976a914904a49878c0adfc3aa05de7afad2cc15f483a56a88ac7f400900000000001976a914418327e3f3dda4cf5b9089325a4b95abdfa0334088ac722c0c00000000001976a914ba35042cfe9fc66fd35ac2224eebdafd1028ad2788acdc4ace020000000017a91474d691da1574e6b3c192ecfb52cc8984ee7b6c568700000000'
stream = BytesIO(bytes.fromhex(hex_tx))

# parse the S256Point and Signature
point = S256Point.parse(sec)
sig = Signature.parse(der)
# parse the Tx
t = Tx.parse(stream)
# change the first input's scriptSig to redeemScript (use Script.parse on the redeemScript)
t.tx_ins[0].script_sig = Script.parse(redeem_script)
# get the serialization
s = t.serialize()
# add the sighash (4 bytes, little-endian of SIGHASH_ALL)
s += int_to_little_endian(SIGHASH_ALL, 4)
# double_sha256 the result
to_sign = double_sha256(s)
# this interpreted is a big-endian number is your z
z = int.from_bytes(to_sign, 'big')
# now verify the signature using point.verify
print(point.verify(z, sig))

True
